<h2 style='text-align: center;'> Technology Capstone Research Project PG (11522) </h2>
<h3 style='text-align: center;'> Project ID: 2024-S1-50 </h3>
<h3 style='text-align: center;'> Group ID: 11522-24S1-41 </h3>
<h4 style='text-align: center;'> Member: Pauline Armamento - U3246782 </h4>
<h4 style='text-align: center;'> Prompt Technique: Few-shot </h4>

### HumanEval

The HumanEval dataset released by OpenAI includes 164 programming problems with a function sig- nature, docstring, body, and several unit tests. They were handwritten to ensure not to be included in the training set of code generation models.
https://huggingface.co/datasets/openai_humaneval

# Task ID 102
Prompt: Write a Python function that takes two positive numbers x and y and returns the biggest even integer number that is in the range [x, y] inclusive. If no even number exists in that range return -1.


### Revised Prompt for Chain Of Thought
Prompt: We want to find the biggest even number within a closed range defined by two positive integers x and y. Let's think step by step: 


1. Define Evenness: An even number is any integer that is divisible by 2 with no remainder. In this case, we're interested in the largest even number that falls within the specified range.

2. Identify Candidate: Since we're looking for an even number, we need to first check if either x or y themselves are even. If both x and y are odd, it implies there are no even numbers within the range.

3. Find Maximum Even: 
If x is even, it becomes a potential candidate for the biggest even number.
If y is also even and greater than x, then y becomes the new candidate for the biggest even number.
4. Return Result:

If a valid even candidate (either x or y) is found, return that number.
If neither x nor y are even, there are no even numbers in the range, so return -1 to indicate this.

In [5]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("openai_humaneval")

# Initialize Task Index
task_index = 102
task_153 = dataset["test"][task_index]

# Access Task Details
task_id = task_153["task_id"]
prompt = task_153["prompt"]
canonical_solution = task_153["canonical_solution"]
test_data = task_153["test"]
entry_point = task_153["entry_point"]

# Display Task Details
print(f"Task ID: {task_id}")
print(f"Prompt: {prompt}")
print(f"Canonical Solution: {canonical_solution}")
print(f"Test Data: {test_data}")
print(f"Entry Point: {entry_point}")


Task ID: HumanEval/102
Prompt: 
def choose_num(x, y):
    """This function takes two positive numbers x and y and returns the
    biggest even integer number that is in the range [x, y] inclusive. If 
    there's no such number, then the function should return -1.

    For example:
    choose_num(12, 15) = 14
    choose_num(13, 12) = -1
    """

Canonical Solution:     if x > y:
        return -1
    if y % 2 == 0:
        return y
    if x == y:
        return -1
    return y - 1

Test Data: def check(candidate):

    # Check some simple cases
    assert candidate(12, 15) == 14
    assert candidate(13, 12) == -1
    assert candidate(33, 12354) == 12354
    assert candidate(5234, 5233) == -1
    assert candidate(6, 29) == 28
    assert candidate(27, 10) == -1

    # Check some edge cases that are easy to work out by hand.
    assert candidate(7, 7) == -1
    assert candidate(546, 546) == 546


Entry Point: choose_num


In [1]:
# Task ID: HumanEval/102
# Function - CoT
# Model 1

import timeit
import pandas as pd

def choose_num1(x, y):
    # Step 1: Define Evenness
    def is_even(n):
        return n % 2 == 0
    
    # Step 2: Identify Candidate
    if is_even(x):
        candidate = x
    elif is_even(y):
        candidate = y
    else:
        return -1  # No even numbers in the range
    
    # Step 3: Find Maximum Even
    if is_even(y) and y > x:
        candidate = y
    
    # Step 4: Return Result
    return candidate

# Test cases
mod1_output1 = choose_num1(12, 15)  # Output: 14
mod1_output2 = choose_num1(13, 12)  # Output: -1
mod1_output3 = choose_num1(33, 12354) # Output: 12354

# Execution Time
execution_time_mod1_output1 = timeit.timeit(lambda: choose_num1(12, 15), number=10000)
print("Execution time Model 1 Output 1:", format(execution_time_mod1_output1, '.5f'))

execution_time_mod1_output2 = timeit.timeit(lambda: choose_num1(13, 12), number=10000)
print("Execution time Model 1 Output 2:", format(execution_time_mod1_output2, '.5f'))

execution_time_mod1_output3 = timeit.timeit(lambda: choose_num1(33, 12354), number=10000)
print("Execution time Model 1 Output 3:", format(execution_time_mod1_output3, '.5f'))

average_time_mod1output = (execution_time_mod1_output1 + execution_time_mod1_output2 + execution_time_mod1_output3) / 3
print("Average Execution Time for Model 1 Output:", format(average_time_mod1output, '.5f'))

def evaluate_list_metrics(expected_output, actual_output):
    if isinstance(expected_output, int) or isinstance(actual_output, int):
        # If either expected_output or actual_output is an integer,
        # convert them to lists for comparison
        expected_output = [expected_output]
        actual_output = [actual_output]

    if len(expected_output) != len(actual_output):
        raise ValueError("Expected and actual output lengths must be the same")

    exact_match_accuracy = 1 if expected_output == actual_output else 0

    true_positives = sum(1 for expected, actual in zip(expected_output, actual_output) if expected == actual)
    false_positives = sum(1 for expected, actual in zip(expected_output, actual_output) if expected == 0 and actual == 1)
    false_negatives = sum(1 for expected, actual in zip(expected_output, actual_output) if expected == 1 and actual == 0)

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    accuracy = sum(1 for expected, actual in zip(expected_output, actual_output) if expected == actual) / len(expected_output)

    return {
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1_score,
        "Accuracy": accuracy,
        "Exact Match": exact_match_accuracy
    }

expected_outputs = [14,-1,12354]
actual_outputs_mod1 = [mod1_output1, mod1_output2, mod1_output3]
print("Expected Output: ", expected_outputs)
print("Actual Output: ", actual_outputs_mod1)

metrics1 = [evaluate_list_metrics(expected, actual) for expected, actual in zip(expected_outputs, actual_outputs_mod1)]

# Create DataFrame directly from the list of dictionaries
df_metrics1_new = pd.DataFrame(metrics1)

# Calculate the average for precision, recall, F1 score, accuracy, and exact match
avg_metrics1 = df_metrics1_new.mean(axis=0)
avg_metrics1.name = 'Model 1 Average'

# Concatenate the average row to the DataFrame
df_metrics1_new = pd.concat([df_metrics1_new, avg_metrics1.to_frame().transpose()])

# Print the new DataFrame
print(df_metrics1_new)



Execution time Model 1 Output 1: 0.00330
Execution time Model 1 Output 2: 0.00463
Execution time Model 1 Output 3: 0.00449
Average Execution Time for Model 1 Output: 0.00414
Expected Output:  [14, -1, 12354]
Actual Output:  [12, 12, 12354]
                 Precision    Recall  F1 Score  Accuracy  Exact Match
0                 0.000000  0.000000  0.000000  0.000000     0.000000
1                 0.000000  0.000000  0.000000  0.000000     0.000000
2                 1.000000  1.000000  1.000000  1.000000     1.000000
Model 1 Average   0.333333  0.333333  0.333333  0.333333     0.333333


In [2]:
# Task ID: HumanEval/102
# Function - CoT
# Model 2


def choose_num2(x, y):
    # Step 1: Define Evenness
    def is_even(n):
        return n % 2 == 0
    
    # Step 2: Find Upper Limit
    upper_limit = y if is_even(y) else y - 1
    
    # Step 3: Check Validity
    if upper_limit < x:
        return -1
    else:
        return upper_limit
    
# Test cases
mod2_output1 = choose_num2(12, 15)  # Output: 14
mod2_output2 = choose_num2(13, 12)  # Output: -1
mod2_output3 = choose_num2(33, 12354) # Output: 12354

# Execution Time
execution_time_mod2_output1 = timeit.timeit(lambda: choose_num2(12, 15), number=10000)
print("Execution time Model 2 Output 1:", format(execution_time_mod2_output1, '.5f'))

execution_time_mod2_output2 = timeit.timeit(lambda: choose_num2(13, 12), number=10000)
print("Execution time Model 2 Output 2:", format(execution_time_mod2_output2, '.5f'))

execution_time_mod2_output3 = timeit.timeit(lambda: choose_num2(33, 12354), number=10000)
print("Execution time Model 2 Output 3:", format(execution_time_mod2_output3, '.5f'))

average_time_mod2output = (execution_time_mod2_output1 + execution_time_mod2_output2 + execution_time_mod2_output3) / 3
print("Average Execution Time for Model 2 Output:", format(average_time_mod2output, '.5f'))

def evaluate_list_metrics(expected_output, actual_output):
    if isinstance(expected_output, int) or isinstance(actual_output, int):
        # If either expected_output or actual_output is an integer,
        # convert them to lists for comparison
        expected_output = [expected_output]
        actual_output = [actual_output]

    if len(expected_output) != len(actual_output):
        raise ValueError("Expected and actual output lengths must be the same")

    exact_match_accuracy = 1 if expected_output == actual_output else 0

    true_positives = sum(1 for expected, actual in zip(expected_output, actual_output) if expected == actual)
    false_positives = sum(1 for expected, actual in zip(expected_output, actual_output) if expected == 0 and actual == 1)
    false_negatives = sum(1 for expected, actual in zip(expected_output, actual_output) if expected == 1 and actual == 0)

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    accuracy = sum(1 for expected, actual in zip(expected_output, actual_output) if expected == actual) / len(expected_output)

    return {
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1_score,
        "Accuracy": accuracy,
        "Exact Match": exact_match_accuracy
    }

expected_outputs = [14,-1,12354]
actual_outputs_mod2 = [mod2_output1, mod2_output2, mod2_output3]
print("Expected Output: ", expected_outputs)
print("Actual Output: ", actual_outputs_mod2)

metrics2 = [evaluate_list_metrics(expected, actual) for expected, actual in zip(expected_outputs, actual_outputs_mod2)]

# Create DataFrame directly from the list of dictionaries
df_metrics2_new = pd.DataFrame(metrics2)

# Calculate the average for precision, recall, F1 score, accuracy, and exact match
avg_metrics2 = df_metrics2_new.mean(axis=0)
avg_metrics2.name = 'Model 2 Average'

# Concatenate the average row to the DataFrame
df_metrics2_new = pd.concat([df_metrics2_new, avg_metrics2.to_frame().transpose()])

# Print the new DataFrame
print(df_metrics2_new)



Execution time Model 2 Output 1: 0.00429
Execution time Model 2 Output 2: 0.00372
Execution time Model 2 Output 3: 0.00328
Average Execution Time for Model 2 Output: 0.00376
Expected Output:  [14, -1, 12354]
Actual Output:  [14, -1, 12354]
                 Precision  Recall  F1 Score  Accuracy  Exact Match
0                      1.0     1.0       1.0       1.0          1.0
1                      1.0     1.0       1.0       1.0          1.0
2                      1.0     1.0       1.0       1.0          1.0
Model 2 Average        1.0     1.0       1.0       1.0          1.0


In [3]:
# Task ID: HumanEval/102
# Function - CoT
# Model 3

def choose_num3(x, y):
    # Step 1: Define Evenness
    def is_even(n):
        return n % 2 == 0
    
    # Step 2: Find the Highest Even Number
    for num in range(y, x - 1, -1):
        if is_even(num):
            return num
    
    # Step 3: If no even number is found
    return -1

# Test cases
mod3_output1 = choose_num3(12, 15)  # Output: 14
mod3_output2 = choose_num3(13, 12)  # Output: -1
mod3_output3 = choose_num3(33, 12354) # Output: 12354

# Execution Time
execution_time_mod3_output1 = timeit.timeit(lambda: choose_num3(12, 15), number=10000)
print("Execution time Model 3 Output 1:", format(execution_time_mod3_output1, '.5f'))

execution_time_mod3_output2 = timeit.timeit(lambda: choose_num3(13, 12), number=10000)
print("Execution time Model 3 Output 2:", format(execution_time_mod3_output2, '.5f'))

execution_time_mod3_output3 = timeit.timeit(lambda: choose_num3(33, 12354), number=10000)
print("Execution time Model 3 Output 3:", format(execution_time_mod3_output3, '.5f'))

average_time_mod3output = (execution_time_mod3_output1 + execution_time_mod3_output2 + execution_time_mod3_output3) / 3
print("Average Execution Time for Model 3 Output:", format(average_time_mod3output, '.5f'))

def evaluate_list_metrics(expected_output, actual_output):
    if isinstance(expected_output, int) or isinstance(actual_output, int):
        # If either expected_output or actual_output is an integer,
        # convert them to lists for comparison
        expected_output = [expected_output]
        actual_output = [actual_output]

    if len(expected_output) != len(actual_output):
        raise ValueError("Expected and actual output lengths must be the same")

    exact_match_accuracy = 1 if expected_output == actual_output else 0

    true_positives = sum(1 for expected, actual in zip(expected_output, actual_output) if expected == actual)
    false_positives = sum(1 for expected, actual in zip(expected_output, actual_output) if expected == 0 and actual == 1)
    false_negatives = sum(1 for expected, actual in zip(expected_output, actual_output) if expected == 1 and actual == 0)

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    accuracy = sum(1 for expected, actual in zip(expected_output, actual_output) if expected == actual) / len(expected_output)

    return {
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1_score,
        "Accuracy": accuracy,
        "Exact Match": exact_match_accuracy
    }

expected_outputs = [14,-1,12354]
actual_outputs_mod3 = [mod3_output1, mod3_output2, mod3_output3]
print("Expected Output: ", expected_outputs)
print("Actual Output: ", actual_outputs_mod3)

metrics3 = [evaluate_list_metrics(expected, actual) for expected, actual in zip(expected_outputs, actual_outputs_mod3)]

# Create DataFrame directly from the list of dictionaries
df_metrics3_new = pd.DataFrame(metrics3)

# Calculate the average for precision, recall, F1 score, accuracy, and exact match
avg_metrics3 = df_metrics3_new.mean(axis=0)
avg_metrics3.name = 'Model 3 Average'

# Concatenate the average row to the DataFrame
df_metrics3_new = pd.concat([df_metrics3_new, avg_metrics3.to_frame().transpose()])

# Print the new DataFrame
print(df_metrics3_new)


Execution time Model 3 Output 1: 0.00702
Execution time Model 3 Output 2: 0.00292
Execution time Model 3 Output 3: 0.00546
Average Execution Time for Model 3 Output: 0.00513
Expected Output:  [14, -1, 12354]
Actual Output:  [14, -1, 12354]
                 Precision  Recall  F1 Score  Accuracy  Exact Match
0                      1.0     1.0       1.0       1.0          1.0
1                      1.0     1.0       1.0       1.0          1.0
2                      1.0     1.0       1.0       1.0          1.0
Model 3 Average        1.0     1.0       1.0       1.0          1.0


In [4]:
# Task ID: HumanEval/102
# Overall Metrics

# Calculate the averages for each set of metrics
avg_metrics1 = df_metrics1_new.mean(axis=0)
avg_metrics2 = df_metrics2_new.mean(axis=0)
avg_metrics3 = df_metrics3_new.mean(axis=0)

# Calculate the overall average time
overall_average_time = (average_time_mod1output + average_time_mod2output + average_time_mod3output) / 3

# Create a DataFrame to store the averages
df_avg_metrics = pd.DataFrame([avg_metrics1, avg_metrics2, avg_metrics3])

# Add a new column for average time
df_avg_metrics['Average Time'] = [average_time_mod1output, average_time_mod2output, average_time_mod3output]

# Add a new row displaying overall average
df_avg_metrics.loc['Overall Average'] = df_avg_metrics.mean()

# Add a new row for overall average time
df_avg_metrics.loc['Overall Average Time'] = overall_average_time

# Rename the index to include model names
df_avg_metrics.index = ['Model 1', 'Model 2', 'Model 3', 'Overall Average', 'Overall Average Time']

# Print the DataFrame
print(df_avg_metrics)

                      Precision    Recall  F1 Score  Accuracy  Exact Match  \
Model 1                0.333333  0.333333  0.333333  0.333333     0.333333   
Model 2                1.000000  1.000000  1.000000  1.000000     1.000000   
Model 3                1.000000  1.000000  1.000000  1.000000     1.000000   
Overall Average        0.777778  0.777778  0.777778  0.777778     0.777778   
Overall Average Time   0.004345  0.004345  0.004345  0.004345     0.004345   

                      Average Time  
Model 1                   0.004140  
Model 2                   0.003760  
Model 3                   0.005134  
Overall Average           0.004345  
Overall Average Time      0.004345  
